In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from spark_processing import WEATHER_DATA_SCHEMA

In [33]:
schema_json = WEATHER_DATA_SCHEMA.jsonValue()['fields']
field_types = {row['name']: row['type'] for row in schema_json}

field_types.keys()

dict_keys(['datetimeEpoch', 'temp', 'feelslike', 'humidity', 'dew', 'precip', 'precipprob', 'snow', 'snowdepth', 'preciptype', 'windgust', 'windspeed', 'winddir', 'pressure', 'visibility', 'cloudcover', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'conditions', 'icon', 'stations', 'source', 'time', 'lat', 'lon', 'timezone', 'tzoffset', 'date', 'day_agg_tempmax', 'day_agg_tempmin', 'day_agg_temp', 'day_agg_feelslikemax', 'day_agg_feelslikemin', 'day_agg_feelslike', 'day_agg_dew', 'day_agg_humidity', 'day_agg_precip', 'day_agg_precipprob', 'day_agg_precipcover', 'day_agg_preciptype', 'day_agg_snow', 'day_agg_snowdepth', 'day_agg_windgust', 'day_agg_windspeed', 'day_agg_winddir', 'day_agg_pressure', 'day_agg_cloudcover', 'day_agg_visibility', 'day_agg_uvindex', 'day_agg_severerisk', 'day_agg_sunrise', 'day_agg_sunset', 'day_agg_moonphase', 'day_agg_conditions', 'day_agg_description', 'day_agg_source'])

In [28]:
from vc_weather import collect_api_data

lat = 37.560614
lon = -122.412083

start = "2023-02-23"
end = "2023-02-23"

api_data = collect_api_data(lat, lon, start, end)

In [35]:
def enforce_spark_schema(api_data):
    """ Function to ensure the api dataframe adheres to the expect 
    spark sql schema. Adds missing columns and fills nan values 
    with appriate value for the column dtype """
    # Load spark df schema and extract field names / types
    schema_json = WEATHER_DATA_SCHEMA.jsonValue()['fields']
    field_types = {row['name']: row['type'] for row in schema_json}
    # Func to provide missing value for different dtypes
    nan_type = lambda dtype: 'MISSING' if dtype == 'string' else 0
    # Loop through each field and determine the nan value
    for field, datatype in field_types.items():
        nan_val = nan_type(datatype)
        # If field missing from df add column of nan values
        if field in api_data.columns:
            api_data[field] = api_data[field].fillna(nan_val)
        # Else fill all NaN with nan values
        else:
            api_data[field] = nan_val
    # Return dataframe with correct column ordering
    api_data[field_types.keys()]

,datetimeEpoch,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,preciptype,windgust,windspeed,winddir,pressure,visibility,cloudcover,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,source,time,lat,lon,timezone,tzoffset,date,day_agg_tempmax,day_agg_tempmin,day_agg_temp,day_agg_feelslikemax,day_agg_feelslikemin,day_agg_feelslike,day_agg_dew,day_agg_humidity,day_agg_precip,day_agg_precipprob,day_agg_precipcover,day_agg_preciptype,day_agg_snow,day_agg_snowdepth,day_agg_windgust,day_agg_windspeed,day_agg_winddir,day_agg_pressure,day_agg_cloudcover,day_agg_visibility,day_agg_uvindex,day_agg_severerisk,day_agg_sunrise,day_agg_sunset,day_agg_moonphase,day_agg_conditions,day_agg_description,day_agg_source
0,1677139200,6.5,4.2,71.51,1.7,0.320,100.0,0.0,0.0,rain,27.7,11.3,292.0,1010.8,14.1,92.3,0.0,0.0,0.0,10.0,"Rain, Overcast",rain,KHAF|KSFO|72493023230|72064600228|KOAK|7249402...,obs,00:00:00,37.560614,-122.412083,America/Los_Angeles,-8.0,2023-02-23,9.7,3.2,7.0,6.8,-0.4,3.7,2.4,73.2,6.799,100.0,41.67,rain,0.0,0.0,56.3,38.8,188.9,1009.8,82.2,14.1,5.0,10.0,06:49:11,17:57:23,0.12,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,obs
1,1677142800,5.5,2.6,73.11,1.0,0.000,0.0,0.0,0.0,No rain,33.5,13.1,293.0,1011.0,14.1,35.2,0.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-night,KHAF|KSFO|72493023230|72064600228|KOAK|7249402...,obs,01:00:00,37.560614,-122.412083,America/Los_Angeles,-8.0,2023-02-23,9.7,3.2,7.0,6.8,-0.4,3.7,2.4,73.2,6.799,100.0,41.67,rain,0.0,0.0,56.3,38.8,188.9,1009.8,82.2,14.1,5.0,10.0,06:49:11,17:57:23,0.12,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,obs
2,1677146400,5.0,2.7,80.27,1.8,0.000,0.0,0.0,0.0,No rain,33.5,9.8,259.0,1011.2,14.7,39.0,0.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-night,KHAF|KSFO|KOAK,obs,02:00:00,37.560614,-122.412083,America/Los_Angeles,-8.0,2023-02-23,9.7,3.2,7.0,6.8,-0.4,3.7,2.4,73.2,6.799,100.0,41.67,rain,0.0,0.0,56.3,38.8,188.9,1009.8,82.2,14.1,5.0,10.0,06:49:11,17:57:23,0.12,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,obs
3,1677150000,5.5,3.1,78.78,2.1,0.076,100.0,0.0,0.0,rain,24.1,10.6,230.0,1011.0,11.8,99.5,0.0,0.0,0.0,10.0,"Rain, Overcast",rain,KHAF|KSFO|KOAK,obs,03:00:00,37.560614,-122.412083,America/Los_Angeles,-8.0,2023-02-23,9.7,3.2,7.0,6.8,-0.4,3.7,2.4,73.2,6.799,100.0,41.67,rain,0.0,0.0,56.3,38.8,188.9,1009.8,82.2,14.1,5.0,10.0,06:49:11,17:57:23,0.12,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,obs
4,1677153600,5.0,2.7,80.31,1.9,0.000,0.0,0.0,0.0,No rain,29.5,10.0,227.0,1010.9,13.8,61.3,0.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-night,KHAF|KSFO|KOAK,obs,04:00:00,37.560614,-122.412083,America/Los_Angeles,-8.0,2023-02-23,9.7,3.2,7.0,6.8,-0.4,3.7,2.4,73.2,6.799,100.0,41.67,rain,0.0,0.0,56.3,38.8,188.9,1009.8,82.2,14.1,5.0,10.0,06:49:11,17:57:23,0.12,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,obs
5,1677157200,3.8,1.6,87.09,1.8,0.000,0.0,0.0,0.0,No rain,25.9,8.5,235.0,1011.1,15.2,64.4,0.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-night,KHAF|KSFO|KOAK,obs,05:00:00,37.560614,-122.412083,America/Los_Angeles,-8.0,2023-02-23,9.7,3.2,7.0,6.8,-0.4,3.7,2.4,73.2,6.799,100.0,41.67,rain,0.0,0.0,56.3,38.8,188.9,1009.8,82.2,14.1,5.0,10.0,06:49:11,17:57:23,0.12,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,obs
6,1677160800,3.7,-0.1,87.83,1.9,1.420,100.0,0.0,0.0,rain,22.3,17.0,161.0,1011.5,12.1,96.4,0.0,0.0,0.0,10.0,"Rain, Overcast",rain,KHAF|KSFO|KOAK,obs,06:00:00,37.560614,-122.412083,America/Los_Angeles,-8.0,2023-02-23,9.7,3.2,7.0,6.8,-0.4,3.7,2.4,73.2,6.799,100.0,41.67,rain,0.0,0.0,56.3,38.8,188.9,1009.8,82.2,14.1,5.0,10.0,06:49:11,17:57:23,0.12,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,obs
7,1677164400,3.2,-0.4,87.36,1.3,0.507,100.0,0.0,0.0,rain,22.3,14.1,145.0,1011.6,14.7,59.6,2.0,0.0,0.0,10.0,"Rain, Partially cloudy",rain,KHAF|KSFO|KOAK,obs,07:00:00,

In [3]:
from airflow_dag import collect_weather_api_data_store_in_gcp, process_data_with_spark_store_in_mongo

In [4]:
collect_weather_api_data_store_in_gcp()

[2023-02-25 14:59:38,175] {airflow_dag.py:53} INFO - Processing: lat = 34.114 lon = -118.407 start = '2022-01-01' end = '2022-01-31'


In [5]:
collect_weather_api_data_store_in_gcp()

[2023-02-25 15:00:01,481] {airflow_dag.py:53} INFO - Processing: lat = 34.114 lon = -118.407 start = '2022-02-01' end = '2022-02-28'


In [6]:
process_data_with_spark_store_in_mongo()

23/02/25 15:00:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[2023-02-25 15:00:51,656] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.114_-118.407_2022_01.csv'
23/02/25 15:00:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[2023-02-25 15:01:00,868] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.114_-118.407_2022_02.csv'


In [18]:
from vc_weather import collect_api_data

In [4]:
from airflow_dag import process_data_with_spark_store_in_mongo

In [5]:
process_data_with_spark_store_in_mongo()

[2023-02-24 23:34:14,634] {airflow_dag.py:90} INFO - Processing:	gcp_filename = 'test_data.csv'
23/02/24 23:34:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/24 23:34:26 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [19]:
lat = 37.560614
lon = -122.412083

start = "2023-02-23"
end = "2023-02-23"

api_data = collect_api_data(lat, lon, start, end)

In [29]:
type(api_data.solarenergy.loc[0])

numpy.float64

In [37]:
api_data.fillna(0).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 58 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetimeEpoch         24 non-null     int64  
 1   temp                  24 non-null     float64
 2   feelslike             24 non-null     float64
 3   humidity              24 non-null     float64
 4   dew                   24 non-null     float64
 5   precip                24 non-null     float64
 6   precipprob            24 non-null     float64
 7   snow                  24 non-null     float64
 8   snowdepth             24 non-null     float64
 9   preciptype            24 non-null     object 
 10  windgust              24 non-null     float64
 11  windspeed             24 non-null     float64
 12  winddir               24 non-null     float64
 13  pressure              24 non-null     float64
 14  visibility            24 non-null     float64
 15  cloudcover            24 

In [40]:
api_data[['date', 'time', 'day_agg_sunrise', 'day_agg_sunset']]

,date,time,day_agg_sunrise,day_agg_sunset
0,2023-02-23,00:00:00,06:49:11,17:57:23
1,2023-02-23,01:00:00,06:49:11,17:57:23
2,2023-02-23,02:00:00,06:49:11,17:57:23
3,2023-02-23,03:00:00,06:49:11,17:57:23
4,2023-02-23,04:00:00,06:49:11,17:57:23
5,2023-02-23,05:00:00,06:49:11,17:57:23
6,2023-02-23,06:00:00,06:49:11,17:57:23
7,2023-02-23,07:00:00,06:49:11,17:57:23
8,2023-02-23,08:00:00,06:49:11,17:57:23
9,2023-02-23,09:00:00,06:49:11,17:57:23


In [33]:
(api_data == np.nan).sum()

datetimeEpoch           0
temp                    0
feelslike               0
humidity                0
dew                     0
precip                  0
precipprob              0
snow                    0
snowdepth               0
preciptype              0
windgust                0
windspeed               0
winddir                 0
pressure                0
visibility              0
cloudcover              0
solarradiation          0
solarenergy             0
uvindex                 0
severerisk              0
conditions              0
icon                    0
stations                0
source                  0
time                    0
lat                     0
lon                     0
timezone                0
tzoffset                0
date                    0
day_agg_tempmax         0
day_agg_tempmin         0
day_agg_temp            0
day_agg_feelslikemax    0
day_agg_feelslikemin    0
day_agg_feelslike       0
day_agg_dew             0
day_agg_humidity        0
day_agg_prec

In [31]:
import numpy as np

type(np.nan)

float

In [21]:
api_data[['date','time']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    24 non-null     object
 1   time    24 non-null     object
dtypes: object(2)
memory usage: 512.0+ bytes


In [5]:
api_data[['date', 'time', 'feelslike', 'solarradiation', 'cloudcover']]

,date,time,feelslike,solarradiation,cloudcover
0,2023-02-23,00:00:00,4.2,0.0,92.3
1,2023-02-23,01:00:00,2.6,0.0,35.2
2,2023-02-23,02:00:00,2.7,0.0,39.0
3,2023-02-23,03:00:00,3.1,0.0,99.5
4,2023-02-23,04:00:00,2.7,0.0,61.3
5,2023-02-23,05:00:00,1.6,0.0,64.4
6,2023-02-23,06:00:00,-0.1,0.0,96.4
7,2023-02-23,07:00:00,-0.4,2.0,59.6
8,2023-02-23,08:00:00,1.5,116.0,88.0
9,2023-02-23,09:00:00,3.7,287.0,88.0


In [6]:
from gcp_bucket import GCP_Bucket

In [7]:
bucket = GCP_Bucket()

bucket.upload_file(api_data, 'test_data.csv')

In [7]:
from spark_processing import create_spark_session, read_file_from_gcp

In [8]:
session = create_spark_session()

# session.sparkContext.setLogLevel("INFO")
# df = read_file_from_gcp(session, 'test_data.csv')

In [9]:
df = read_file_from_gcp(session, 'test_data.csv')

In [44]:
df.show()

23/02/24 22:44:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: datetimeEpoch, temp, feelslike, humidity, dew, precip, precipprob, snow, snowdepth, preciptype, windgust, windspeed, winddir, pressure, visibility, cloudcover, solarradiation, solarenergy, uvindex, severerisk, conditions, icon, stations, source, time, lat, lon, timezone, tzoffset, date, day_agg_tempmax, day_agg_tempmin, day_agg_temp, day_agg_feelslikemax, day_agg_feelslikemin, day_agg_feelslike, day_agg_dew, day_agg_humidity, day_agg_precip, day_agg_precipprob, day_agg_precipcover, day_agg_preciptype, day_agg_snow, day_agg_snowdepth, day_agg_windgust, day_agg_windspeed, day_agg_winddir, day_agg_pressure, day_agg_cloudcover, day_agg_visibility, day_agg_uvindex, day_agg_severerisk, day_agg_sunrise, day_agg_sunset, day_agg_moonphase, day_agg_conditions, day_agg_description, day_agg_source
 Schema: datetimeEpoch, temp, feelslike, humidity, dew, precip, precipprob, snow, snowdepth, preciptype, windg

In [17]:
rows = df.select("*").collect()

23/02/24 21:31:00 INFO FileSourceStrategy: Pushed Filters: 
23/02/24 21:31:00 INFO FileSourceStrategy: Post-Scan Filters: 
23/02/24 21:31:00 INFO FileSourceStrategy: Output Data Schema: struct<datetimeEpoch: int, temp: double, feelslike: double, humidity: double, dew: double ... 56 more fields>
23/02/24 21:31:00 INFO MemoryStore: Block broadcast_20 stored as values in memory (estimated size 201.1 KiB, free 433.7 MiB)
23/02/24 21:31:00 INFO MemoryStore: Block broadcast_20_piece0 stored as bytes in memory (estimated size 34.6 KiB, free 433.6 MiB)
23/02/24 21:31:00 INFO BlockManagerInfo: Added broadcast_20_piece0 in memory on localhost:52259 (size: 34.6 KiB, free: 434.3 MiB)
23/02/24 21:31:00 INFO SparkContext: Created broadcast 20 from collect at /var/folders/4p/647j9vhx2js0krnv5kzs32j80000gn/T/ipykernel_6155/3413396465.py:1
23/02/24 21:31:00 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
23/02/24 21:31

In [18]:
rows[0].asDict()

{'datetimeEpoch': 1677139200,
 'temp': 6.5,
 'feelslike': 4.2,
 'humidity': 71.51,
 'dew': 1.7,
 'precip': 0.32,
 'precipprob': 100.0,
 'snow': 0.0,
 'snowdepth': 0.0,
 'preciptype': 'rain',
 'windgust': 27.7,
 'windspeed': 11.3,
 'winddir': 292.0,
 'pressure': 1010.8,
 'visibility': 14.1,
 'cloudcover': 92.3,
 'solarradiation': 0.0,
 'solarenergy': None,
 'uvindex': 0.0,
 'severerisk': 10.0,
 'conditions': 'Rain, Overcast',
 'icon': 'rain',
 'stations': 'KHAF|KSFO|72493023230|72064600228|KOAK|72494023234',
 'source': 'obs',
 'time': datetime.datetime(2023, 2, 24, 0, 0),
 'lat': 37.560614,
 'lon': -122.412083,
 'timezone': 'America/Los_Angeles',
 'tzoffset': -8.0,
 'date': datetime.datetime(2023, 2, 23, 0, 0),
 'day_agg_tempmax': 9.7,
 'day_agg_tempmin': 3.2,
 'day_agg_temp': 7.0,
 'day_agg_feelslikemax': 6.8,
 'day_agg_feelslikemin': -0.4,
 'day_agg_feelslike': 3.7,
 'day_agg_dew': 2.4,
 'day_agg_humidity': 73.2,
 'day_agg_precip': 6.799,
 'day_agg_precipprob': 100.0,
 'day_agg_precip

23/02/24 21:51:56 INFO BlockManagerInfo: Removed broadcast_17_piece0 on localhost:52259 in memory (size: 14.6 KiB, free: 434.3 MiB)
23/02/24 21:51:56 INFO BlockManagerInfo: Removed broadcast_18_piece0 on localhost:52259 in memory (size: 34.6 KiB, free: 434.3 MiB)
23/02/24 21:51:56 INFO BlockManagerInfo: Removed broadcast_16_piece0 on localhost:52259 in memory (size: 34.6 KiB, free: 434.4 MiB)
23/02/24 21:51:56 INFO BlockManagerInfo: Removed broadcast_21_piece0 on localhost:52259 in memory (size: 7.7 KiB, free: 434.4 MiB)
23/02/24 21:51:56 INFO BlockManagerInfo: Removed broadcast_19_piece0 on localhost:52259 in memory (size: 7.8 KiB, free: 434.4 MiB)


In [10]:
df.printSchema()

root
 |-- datetimeEpoch: integer (nullable = true)
 |-- temp: double (nullable = true)
 |-- feelslike: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- dew: double (nullable = true)
 |-- precip: double (nullable = true)
 |-- precipprob: double (nullable = true)
 |-- snow: double (nullable = true)
 |-- snowdepth: double (nullable = true)
 |-- preciptype: string (nullable = true)
 |-- windgust: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- winddir: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- visibility: double (nullable = true)
 |-- cloudcover: double (nullable = true)
 |-- solarradiation: double (nullable = true)
 |-- solarenergy: double (nullable = true)
 |-- uvindex: double (nullable = true)
 |-- severerisk: double (nullable = true)
 |-- conditions: string (nullable = true)
 |-- icon: string (nullable = true)
 |-- stations: string (nullable = true)
 |-- source: string (nullable = true)
 |-- time: string (nullable 

In [11]:
df.select('date', 'time', 'day_agg_sunset', 'day_agg_sunrise', 'datetimeEpoch', 'day_agg_cloudcover', 'timezone').show()

+----------+--------+--------------+---------------+-------------+------------------+-------------------+
|      date|    time|day_agg_sunset|day_agg_sunrise|datetimeEpoch|day_agg_cloudcover|           timezone|
+----------+--------+--------------+---------------+-------------+------------------+-------------------+
|2023-02-23|00:00:00|      17:57:23|       06:49:11|   1677139200|              82.2|America/Los_Angeles|
|2023-02-23|01:00:00|      17:57:23|       06:49:11|   1677142800|              82.2|America/Los_Angeles|
|2023-02-23|02:00:00|      17:57:23|       06:49:11|   1677146400|              82.2|America/Los_Angeles|
|2023-02-23|03:00:00|      17:57:23|       06:49:11|   1677150000|              82.2|America/Los_Angeles|
|2023-02-23|04:00:00|      17:57:23|       06:49:11|   1677153600|              82.2|America/Los_Angeles|
|2023-02-23|05:00:00|      17:57:23|       06:49:11|   1677157200|              82.2|America/Los_Angeles|
|2023-02-23|06:00:00|      17:57:23|       06:

In [17]:
time = df.select("*").collect()[1].asDict()['time']
time

datetime.datetime(2023, 2, 24, 1, 0)

In [16]:
from pyspark.sql.functions import isnull

null_count = 0

for col in df.columns:
    x = df.filter(isnull(col)).count()
    null_count += x
    if x > 0:
        print(col)

solarenergy


In [15]:
null_count

8

In [13]:
import pyspark
print(pyspark.__version__)

3.3.0


In [18]:
[row.asDict() for row in df.collect()]

[{'datetimeEpoch': 1677139200,
  'temp': 6.5,
  'feelslike': 4.2,
  'humidity': 71.51,
  'dew': 1.7,
  'precip': 0.32,
  'precipprob': 100.0,
  'snow': 0.0,
  'snowdepth': 0.0,
  'preciptype': 'rain',
  'windgust': 27.7,
  'windspeed': 11.3,
  'winddir': 292.0,
  'pressure': 1010.8,
  'visibility': 14.1,
  'cloudcover': 92.3,
  'solarradiation': 0.0,
  'solarenergy': None,
  'uvindex': 0.0,
  'severerisk': 10.0,
  'conditions': 'Rain, Overcast',
  'icon': 'rain',
  'stations': 'KHAF|KSFO|72493023230|72064600228|KOAK|72494023234',
  'source': 'obs',
  'time': '00:00:00',
  'lat': 37.560614,
  'lon': -122.412083,
  'timezone': 'America/Los_Angeles',
  'tzoffset': -8.0,
  'date': datetime.date(2023, 2, 23),
  'day_agg_tempmax': 9.7,
  'day_agg_tempmin': 3.2,
  'day_agg_temp': 7.0,
  'day_agg_feelslikemax': 6.8,
  'day_agg_feelslikemin': -0.4,
  'day_agg_feelslike': 3.7,
  'day_agg_dew': 2.4,
  'day_agg_humidity': 73.2,
  'day_agg_precip': 6.799,
  'day_agg_precipprob': 100.0,
  'day_agg_p